## This file contains the code of producing results/figures in Section 5.2 and 5.3


In [ ]:
import numpy as np
import pickle
from datascience import *

import matplotlib
%matplotlib inline   
import matplotlib.pyplot as plt
import time


#=========== Default Paramter Values ================

payment = 10   # everage product profit + fixed rate, added on 5/20/2021

c_s = 2 # USD # cost of moving a car for one km


c_t = ( 22.93 / (5 * 1.6) + 0.443 ) * 0.823  #2.73 $/km，  cost of moving a truck for one km

c_f =  200 # consider a bigger number 
s_c0 = 10  # moving speed of a car, km / hr, replaced on Feb 26, 2022 to be FSA specific 
eta = 0.1  # the average volume of products purchased by a customer
V = 3  # the average vacant space of a stall to fill up upon replenishment
half_day = 12

mu = 60/5/2
sigma_2 = 0  # standard deviation of the time of serving a customer

alpha_0 = 1   
alpha_1 = 1   # wait effect
#alpha_1 = 2   # double alpha 1
alpha_2 = 3   # walking effect
#alpha_2 = 6   # doubled

rho_ub = 2
rho_lb = 0.2

radius_lb = 0.15
radius_ub = 0.5


#=========================================================


High_Res = True
if High_Res == True:
    num_rho = 20
    num_radius = 35
    num_t = 20
    tol_gap = 0.001
else:
    num_rho = 10
    num_radius = 10
    num_t = 8
    tol_gap = 0.01



## Part 1: On-Demand 

In [ ]:
def f_mean_repo_length(rho_x, radius):
    
    R = 1/np.sqrt(rho_x)
    sqrt2 = np.sqrt(2)
    numerator = 7*sqrt2*(R**3) - 22*(R**2)*radius + 2*sqrt2*R*(radius**2) + 12*(radius**3)

    mean_repo = numerator/15/(R**2)
    
    return mean_repo


def f_var_repo_length(rho_x, radius):
    
    if radius> 1/np.sqrt(2*rho_x):
        print('Too big raidus!!')
        return
    
    R = 1/np.sqrt(rho_x)
    sqrt2 = np.sqrt(2)
    
    numerator = 22*(R**6) -22*sqrt2*(R**5)*radius - 140*sqrt2*(R**3)*(radius**3) + \
                400*(R**2)*(radius**4) - 48*sqrt2*R*(radius**5) -144*(radius**6)

    var_repo = numerator/225/(R**4)
    
    return var_repo



def wait_time_mg1_radius(lamb, rho_x, radius, t='NA', E_T='NA', Var_T='NA',s_c=s_c0):
    
    mean_repo = f_mean_repo_length(rho_x, radius)

    if E_T=='NA' or Var_T=='NA':
        E_T = mean_repo/s_c + 1/mu
        Var_T = f_var_repo_length(rho_x, radius)/(s_c**2) + sigma_2**2
                
    Wait = lamb/2/rho_x * (Var_T+E_T**2)/(1-lamb/rho_x*E_T)
    Wait_actual = Wait + mean_repo/s_c  # added on Feb 28, 2022
    
    if Wait < 0:
        return 'Negative Wait'
        
    return Wait_actual

## Part 2: Demand Pooling

In [ ]:
def f_repo_per_hr_pooling(rho_x, radius, lamb, t, s_c):
    
    la = lamb*2*(radius**2)   # arrival rate in a sub-zone
    b = 1/mu   # deterministic customer service time

    N = 1/rho_x/(2*radius**2)  # number of sub-zones
    l_N = 2*radius*N
    E_Tr = l_N/s_c
    
    E_tau = t + b -1/la*(1-np.exp(-la*b))
    
    repo_per_hr = l_N / (E_Tr+N*E_tau)
    
    return repo_per_hr

def wait_time_pooling(lamb, rho_x, radius, t='NA', E_T='NA', Var_T='NA', s_c=s_c0):
    
    la = lamb*2*(radius**2)   # arrival rate in a sub-zone
    
    b = 1/mu   # deterministic customer service time
    
    N = 1/rho_x/(2*radius**2)  # number of sub-zones
    E_Tr = 2*radius*N/s_c
    E_Tr2 = 4*(radius**2)*(N**2)/(s_c**2)
    
    E_tau = t + b -1/la*(1-np.exp(-la*b))
    E_tau2 = (t+b)**2 - 2/la*((t+b)-t*np.exp(-la*b)) + 2/(la**2)*(1-np.exp(-la*b)) 
    
    h_t = E_Tr2 + N*(N-1)*(E_tau**2) + N*E_tau2 + 2*(E_Tr-t)*(N*E_tau-t) - t**2
    
    wait = h_t / 2 / (E_Tr + N*E_tau)
    
    return wait


lamb0 = 100
rho_x0 = 4
radius0 = 0.2

num_array = 20
scales = np.linspace(0.5, 5, num_array)
wait_times = np.zeros(num_array)

for i in range(num_array):
    t=1/mu * scales[i]
    wait_times[i] = wait_time_pooling(lamb0, rho_x0, radius0, t, s_c=s_c0)

plt.plot(scales, wait_times, '*')
plt.ylabel('Mean Wait Time')
_ = plt.xlabel('t \mu')
    
for i in range(num_array):
    lamb=lamb0 * scales[i]
    wait_times[i] = wait_time_pooling(lamb, rho_x0, radius0, 1.5/mu, s_c=s_c0)
    
    
plt.plot(scales, wait_times, '^')
plt.ylabel('Mean Wait Time')
_ = plt.xlabel('lamb')

## Endogenous $\lambda$

In [ ]:
def lamb_endo(f_x_half_day, rho_x, radius, mode = 'on demand', t='NA', E_T='NA', Var_T='NA', lamb_ub0 = 1000, s_c=s_c0):
    '''Solve for the endogenous demand arrival rate
    
    For a given (rho_x, radius), we need to first solve an equation for Lamb_endo.
    There is no closed-formula for Lamb_endo, because Lamb_endo depends on the waiting time,
    but the waiting time in turn depends on lamb.'''
    
    if mode=='on demand':
        wait_time_fun = wait_time_mg1_radius
    elif mode == 'pooling':
        wait_time_fun = wait_time_pooling
        
    precision=5

    lamb_potential = f_x_half_day/half_day
    
    lamb_lb = 0.000001
    lamb_now = lamb_lb
    lamb_ub = lamb_ub0
    gap = 100    
    
    while gap > tol_gap:
        
        wait_now = wait_time_fun(lamb_now, rho_x,radius, t=t, E_T=E_T, Var_T=Var_T, s_c=s_c)
        if wait_now<0:
            print('Negative Wait') # this is impossible for within-in radius, but possible for pooling
        numerator = np.exp(alpha_0 - alpha_1*wait_now - alpha_2*radius)
        prob_select = numerator / (1+numerator)
        lamb_next = lamb_potential*prob_select
        
        lamb_larger = max(lamb_next, lamb_now)
        lamb_smaller = min(lamb_next, lamb_now)
        
        lamb_lb = max(lamb_smaller, lamb_lb)
        lamb_ub = min(lamb_larger, lamb_ub)
            
        gap = abs(lamb_ub-lamb_lb)/lamb_lb
        
        lamb_temp = lamb_lb * 0.5 + lamb_ub * 0.5
       
        # in case we get a negative wait time...
        wait_temp = wait_time_fun(lamb_temp, rho_x,radius, t=t, E_T=E_T, Var_T=Var_T, s_c=s_c)
        while wait_temp == 'Negative Wait':  # then I want to decrease lamb a little bit
            lamb_temp = np.random.rand()*lamb_temp
            wait_temp = wait_time_fun(lamb_temp, rho_x,radius, t=t, E_T=E_T, Var_T=Var_T, s_c=s_c)         
            
        lamb_now = lamb_temp 

    return lamb_lb, wait_time_fun(lamb_lb, rho_x,radius, t, s_c=s_c)



## Solve On-Demand 

In [ ]:
def eval_obj_stationary(f_x, rho_x):
    
    
    lamb_potential = f_x[0]/half_day
    
    radius = 1/np.sqrt(2*rho_x)
    wait = 0
    
    numerator = np.exp(alpha_0 - alpha_1*wait - alpha_2*radius)
    prob_select = numerator / (1+numerator)
    lamb_peak = lamb_potential*prob_select
     
    lamb_offpeak = lamb_peak  
    
    f_tol = lamb_peak*half_day + lamb_offpeak*half_day
    
    term0 = payment * f_tol   # income
    
    term1 = 0
        
    term2 = c_f * rho_x
    
    term3 = np.sqrt(2)*c_t*f_tol * eta / V / np.sqrt(rho_x)
        
    return term0 - term1 - term2 - term3, lamb_peak



def eval_obj_mg1_radius(f_x, rho_x, radius, s_c):
    
    E_T = f_mean_repo_length(rho_x, radius)/s_c + 1/mu
    Var_T = f_var_repo_length(rho_x, radius)/(s_c**2) + sigma_2**2
    
    lamb_peak, wait = lamb_endo(f_x[0], rho_x, radius, E_T = E_T,  Var_T = Var_T, s_c=s_c)
    
    lamb_offpeak = lamb_peak  
    
    f_tol = lamb_peak*half_day + lamb_offpeak*half_day
    
    term0 = payment * f_tol   # income
    
    term1 = c_s * f_tol * f_mean_repo_length(rho_x, radius)
        
    term2 = c_f * rho_x
    
    term3 = np.sqrt(2*c_s*c_t)*f_tol * eta / V / np.sqrt(rho_x)
        
    return term0 - term1 - term2 - term3, lamb_peak, wait


def solve_mg1_radius(f_x, s_c):

    obj_array = np.zeros([num_rho, num_radius])
    rho_array = np.linspace(rho_lb, rho_ub, num_rho)
    radius_array = np.linspace(0, radius_ub, num_radius)
    lamb_array = -555*np.ones([num_rho, num_radius])
    wait_array = -555*np.ones([num_rho, num_radius])
        
    for i in range(num_rho):
        for j in range(num_radius):
            if radius_array[j] > 1/np.sqrt(2*rho_array[i]):  # impossible case
                obj_array[i,j] = -99
            else:
                obj_array[i,j], lamb_array[i,j], wait_array[i,j] =  eval_obj_mg1_radius(f_x, rho_array[i], radius_array[j], s_c)   

            
    index_opt = np.unravel_index(obj_array.argmax(),obj_array.shape)
    
    row_opt = index_opt[0]
    col_opt = index_opt[1]
    
    if row_opt == 0:
        print('LB of rho binding!')
    elif row_opt == num_rho-1:
        print('UB of rho binding!')
        
    if col_opt == 0:
        print('LB of radius binding!')
    elif radius_array[col_opt] > 1/np.sqrt(2*rho_array[row_opt]) * (num_radius-1)/num_radius:
        print('UB of radius binding!')
     
    rho_opt = rho_array[row_opt]
    radius_opt = radius_array[col_opt]
    obj_opt = obj_array[row_opt, col_opt]
    lamb_endo = lamb_array[row_opt, col_opt]
    wait_endo = wait_array[row_opt, col_opt]
    
    Solution = {'rho_opt':rho_opt, 'radius_opt':radius_opt, 'lamb_endo':lamb_endo, 'wait_endo':wait_endo, 'obj_opt':obj_opt}
    
    return Solution, obj_array


def solve_stationary(f_x):
    
    obj_array = np.zeros(num_rho)
    rho_array = np.linspace(rho_lb, rho_ub, num_rho)
    lamb_array = -555*np.ones(num_rho)

    for i in range(num_rho):
        obj_array[i], lamb_array[i] =  eval_obj_stationary(f_x, rho_array[i])   

    
    index_opt = np.unravel_index(obj_array.argmax(),obj_array.shape)
    rho_opt = rho_array[index_opt]
    obj_opt = obj_array[index_opt]
    lamb_endo = lamb_array[index_opt]
    
    Solution = {'rho_opt':rho_opt, 'radius_opt':1/np.sqrt(2*rho_opt), 'lamb_endo':lamb_endo, 'wait_endo':0, 'obj_opt':obj_opt}
    
    return Solution, obj_array


def solve_mg1_d2d(f_x, s_c):

    obj_array = np.zeros(num_rho)
    rho_array = np.linspace(rho_lb, rho_ub, num_rho)
    lamb_array = -555*np.ones(num_rho)
    wait_array = -555*np.ones(num_rho)
        
    for i in range(num_rho):
        obj_array[i], lamb_array[i], wait_array[i] =  eval_obj_mg1_radius(f_x, rho_array[i], 0, s_c)   

            
    index_opt = np.unravel_index(obj_array.argmax(),obj_array.shape)
     
    rho_opt = rho_array[index_opt]
    obj_opt = obj_array[index_opt]
    lamb_endo = lamb_array[index_opt]
    wait_endo = wait_array[index_opt]
    
    Solution = {'rho_opt':rho_opt, 'lamb_endo':lamb_endo, 'wait_endo':wait_endo, 'obj_opt':obj_opt}
    
    return Solution, obj_array

## Solve Pooling 

In [ ]:

def eval_obj_pooling(f_x, rho_x, radius, tt, s_c):
    
    lamb_peak, wait_peak = lamb_endo(f_x[0], rho_x, radius, mode = 'pooling', t=tt[0],  lamb_ub0 = 1000, s_c=s_c)
    
    lamb_offpeak = lamb_peak 
    tt[1]=tt[0]

    f_tol = lamb_peak*half_day + lamb_offpeak*half_day
    
    term0 = payment * f_tol   # income
    
    term1 = c_s * (f_repo_per_hr_pooling(rho_x, radius, lamb_peak, tt[0], s_c)*half_day + 
                   f_repo_per_hr_pooling(rho_x, radius, lamb_offpeak, tt[1], s_c) * half_day) 
        
    term2 = c_f * rho_x
    
    term3 = np.sqrt(2*c_s*c_t)*f_tol * eta / V / np.sqrt(rho_x)
        
    return term0 - term1 - term2 - term3, lamb_peak, wait_peak

def solve_pooling(f_x, s_c):
    
    t_lb = 1/mu
    t_ub = 1

    obj_array = -66*np.ones([num_rho, num_radius, num_t, num_t])
    rho_array = np.linspace(rho_lb, rho_ub, num_rho)
    radius_array = np.linspace(radius_lb, radius_ub, num_radius)
    t_array = np.linspace(t_lb, t_ub, num_t)
    
    lamb_array = -55*np.ones([num_rho, num_radius, num_t, num_t])
    wait_array = -55*np.ones([num_rho, num_radius, num_t, num_t])   
    
    for i in range(num_rho):
        for j in range(num_radius):
            if radius_array[j] > 1/np.sqrt(2*rho_array[i]):  # impossible case
                obj_array[i,j,:,:] = -99
                continue
            else:
                for k1 in range(num_t):
                    for k2 in [0]: 
                        if t_array[k2] > t_array[k1]:   # offpeak t can never be longer than the peak t
                            obj_array[i,j,k1,k2] = -77
                        else:
                            tt = [t_array[k1],t_array[k2]]
                            obj_array[i,j,k1,k2], lamb_array[i,j,k1,k2], wait_array[i,j,k1,k2] =  eval_obj_pooling(f_x, rho_array[i], radius_array[j], tt, s_c)
                            

            
    index_opt = np.unravel_index(obj_array.argmax(),obj_array.shape)
    
    index0_opt = index_opt[0]
    index1_opt = index_opt[1]
    index2_opt = index_opt[2]
    index3_opt = index_opt[3]
     
    rho_opt = rho_array[index0_opt]
    radius_opt = radius_array[index1_opt]
    t0_opt = t_array[index2_opt]
    t1_opt = t_array[index3_opt]
    
        
    obj_opt = obj_array[index_opt]
    lamb_endo = lamb_array[index_opt]
    wait_endo = wait_array[index_opt]
    

    
    Solution = {'rho_opt':rho_opt, 'radius_opt': radius_opt, 't0_opt': t0_opt, 't1_opt': t1_opt, 
                'lamb_endo': lamb_endo,
                'wait_endo': wait_endo,
                'obj_opt':obj_opt}
    
    return Solution, obj_array

## Visualization of Demand Scaling

In [ ]:
num = 20
f_x_values = np.linspace(60,600,num)


rho_stationary_array = np.zeros(num)
radius_stationary_array = np.zeros(num)
obj_stationary_array = np.zeros(num)
lamb_endo_stationary_array = np.zeros(num)
wait_endo_stationary_array = np.zeros(num)


rho_d2d_array = np.zeros(num)
obj_d2d_array = np.zeros(num)
lamb_endo_d2d_array = np.zeros(num)
wait_endo_d2d_array = np.zeros(num)


rho_mg1_array = np.zeros(num)
radius_mg1_array = np.zeros(num)
obj_mg1_array = np.zeros(num)
lamb_endo_mg1_array = np.zeros(num)
wait_endo_mg1_array = np.zeros(num)


rho_pooling_array = np.zeros(num)
radius_pooling_array = np.zeros(num)
t_pooling_array = np.zeros(num)
obj_pooling_array = np.zeros(num)
lamb_endo_pooling_array = np.zeros(num)
wait_endo_pooling_array = np.zeros(num)


#======= stationary =============


for i in range(num):
    
    f_x_now = (f_x_values[i], f_x_values[i])
    
    Solution, obj_array = solve_stationary(f_x_now)
    
    rho_stationary_array[i] = Solution['rho_opt']
    radius_stationary_array[i] = Solution['radius_opt']
    obj_stationary_array[i] = Solution['obj_opt']
    lamb_endo_stationary_array[i] = Solution['lamb_endo']
    wait_endo_stationary_array[i] = Solution['wait_endo']




opt_solution, obj_array = solve_stationary(f_x)

#======= d2d =============


for i in range(num):
    
    f_x_now = (f_x_values[i], f_x_values[i])
    
    Solution, obj_array = solve_mg1_d2d(f_x_now, s_c0)
    
    rho_d2d_array[i] = Solution['rho_opt']
    obj_d2d_array[i] = Solution['obj_opt']
    lamb_endo_d2d_array[i] = Solution['lamb_endo']
    wait_endo_d2d_array[i] = Solution['wait_endo']




#======= Within Radius =============


for i in range(num):
    
    f_x_now = (f_x_values[i], f_x_values[i])
    
    Solution, obj_array = solve_mg1_radius(f_x_now, s_c0)
    
    rho_mg1_array[i] = Solution['rho_opt']
    radius_mg1_array[i] = Solution['radius_opt']
    obj_mg1_array[i] = Solution['obj_opt']
    lamb_endo_mg1_array[i] = Solution['lamb_endo']
    wait_endo_mg1_array[i] = Solution['wait_endo']



#======= Pooling ===============
    

for i in range(num):
        
    f_x_now = (f_x_values[i], f_x_values[i])

    Solution, obj_array = solve_pooling(f_x_now, s_c0)
    
    rho_pooling_array[i] = Solution['rho_opt']
    radius_pooling_array[i] = Solution['radius_opt']
    t_pooling_array[i] = Solution['t0_opt']    
    obj_pooling_array[i] = Solution['obj_opt']
    lamb_endo_pooling_array[i] = Solution['lamb_endo']
    wait_endo_pooling_array[i] = Solution['wait_endo']
    





In [ ]:
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(16, 14)

plt.subplot(3, 2, 1)
plt.plot(f_x_values/half_day, obj_stationary_array, 'm-.', label='Opt stationary')
plt.plot(f_x_values/half_day, obj_d2d_array, 'k--', label='Opt d2d')
plt.plot(f_x_values/half_day, obj_mg1_array, 'b-', label='Opt Within Radius')
plt.plot(f_x_values/half_day, obj_pooling_array, 'r-*', label='Opt Pooling')

plt.xlabel('lambda0')
plt.legend()

plt.subplot(3, 2, 2)
plt.plot(f_x_values/half_day, lamb_endo_stationary_array, 'm-.', label='Lamb Endo stationary')
plt.plot(f_x_values/half_day, lamb_endo_d2d_array, 'k--', label='Lamb Endo d2d')
plt.plot(f_x_values/half_day, lamb_endo_mg1_array, 'b-', label='Lamb Endo Within Radius')
plt.plot(f_x_values/half_day, lamb_endo_pooling_array, 'r-*', label='Lamb Endo Pooling')

plt.xlabel('lambda0')
plt.legend()

plt.subplot(3, 2, 3)
plt.plot(f_x_values/half_day, wait_endo_stationary_array, 'm-.', label='wait_endo stationary')
plt.plot(f_x_values/half_day, wait_endo_d2d_array, 'k--', label='wait_endo d2d')
plt.plot(f_x_values/half_day, wait_endo_mg1_array, 'b-', label='wait_endo Within Radius')
plt.plot(f_x_values/half_day, wait_endo_pooling_array, 'r-*', label='wait_endo Pooling')

plt.xlabel('lambda0')
plt.legend()


plt.subplot(3, 2, 4)
plt.plot(f_x_values/half_day, rho_stationary_array, 'm-.', label='rho stationary')
plt.plot(f_x_values/half_day, rho_d2d_array, 'k--', label='rho d2d')
plt.plot(f_x_values/half_day, rho_mg1_array, 'b-', label='rho Within Radius')
plt.plot(f_x_values/half_day, rho_pooling_array, 'r-*', label='rho Pooling')

plt.xlabel('lambda0')
plt.legend()


plt.subplot(3, 2, 5)
plt.plot(f_x_values/half_day, radius_stationary_array, 'm-.', label='radius Within Radius')
plt.plot(f_x_values/half_day, radius_mg1_array, 'b-', label='radius Within Radius')
plt.plot(f_x_values/half_day, radius_pooling_array, 'r-*', label='radius Pooling')

plt.xlabel('lambda0')
plt.legend()

In [ ]:
Result_Summary = Table().with_columns('lambda', f_x_values/half_day,
                                    'obj_stationary_array',obj_stationary_array,
                                    'obj_d2d_array',obj_d2d_array,
                                    'obj_mg1_array',obj_mg1_array,
                                    'obj_pooling_array',obj_pooling_array,
                                    'rho_stationary_array',rho_stationary_array,
                                    'rho_d2d_array',rho_d2d_array,
                                    'rho_mg1_array',rho_mg1_array,
                                    'rho_pooling_array',rho_pooling_array,
                                    'wait_endo_stationary_array',wait_endo_stationary_array,
                                    'wait_endo_d2d_array',wait_endo_d2d_array,
                                    'wait_endo_mg1_array',wait_endo_mg1_array,
                                    'wait_endo_pooling_array',wait_endo_pooling_array,
                                    'radius_stationary_array',radius_stationary_array, 
                                    'radius_mg1_array',radius_mg1_array,
                                    'radius_pooling_array',radius_pooling_array)

Result_Summary.to_csv('Result_Summary_finer.csv')

# Toronto Simulation

In [ ]:
zipcode_toronto = Table.read_table('zip_codes_toronto.csv').with_column('ID', np.arange(36))
canada_population = Table.read_table('canada_population.csv')
zipcode_pop_toronto = zipcode_toronto.join('zipcode',canada_population,'Geographic code').select('ID','zipcode','Population, 2016')
toronto_areas = Table.read_table('toronto_areas.csv')
zipcode_pop_toronto = zipcode_pop_toronto.join('zipcode', toronto_areas, 'zipcode')
area_numeric = zipcode_pop_toronto.apply(float, 'area')
zipcode_pop_toronto = zipcode_pop_toronto.drop('area').with_columns('area', area_numeric)
pop_density = zipcode_pop_toronto.apply(lambda x,y: x/y, 'Population, 2016', 'area')
zipcode_pop_toronto = zipcode_pop_toronto.with_columns('pop density', pop_density).sort('ID')
zipcode_pop_toronto

## Biking Speed in Toronto (added in Feb 2022)

In [ ]:
toronto_travel_time = Table.read_table('cleaned_toronto_travel_time.csv')
toronto_travel_time_simple = toronto_travel_time.select('FSA','Departure',
                                                        'walking_dist (m)', 'walking_time (s)',
                                                        'biking_dist (m)', 'biking_time (s)',
                                                       'driving_dist (m)', 'driving_time (s)')
toronto_travel_time_simple

In [ ]:
walking_speeds = toronto_travel_time_simple.column('walking_dist (m)') / toronto_travel_time_simple.column('walking_time (s)') * 3600 / 1000
driving_speeds = toronto_travel_time_simple.column('driving_dist (m)') / toronto_travel_time_simple.column('driving_time (s)') * 3600 / 1000
biking_speeds = toronto_travel_time_simple.column('biking_dist (m)') / toronto_travel_time_simple.column('biking_time (s)') * 3600 / 1000
toronto_travel_time_speeds = toronto_travel_time_simple.with_columns('walking_speed', walking_speeds,
                                                                     'biking_speed', biking_speeds,
                                                                    'driving_speed', driving_speeds)
toronto_travel_time_speeds

In [ ]:
mean_speeds = toronto_travel_time_speeds.group(['FSA', 'Departure'], np.mean).drop(2,3,4,5,6,7)
mean_speeds_FSA = mean_speeds.group('FSA', np.mean).select('FSA','biking_speed mean mean').relabeled('biking_speed mean mean','speed')
zipcode_pop_speed_toronto = mean_speeds_FSA.join('FSA', zipcode_pop_toronto,'zipcode').sort('ID')
zipcode_pop_speed_toronto.to_csv('zipcode_pop_speed_toronto.csv')

In [ ]:
speeds = mean_speeds_FSA.column('speed')
s_c_array = zipcode_pop_speed_toronto.column('speed')

In [ ]:
num_locations = 36

demand_scale = 0.05

f_x_values_peak = zipcode_pop_toronto.column('pop density')*demand_scale
f_x_values_offpeak = f_x_values_peak

rho_mg1_array = np.zeros(num_locations)
radius_mg1_array = np.zeros(num_locations)
obj_mg1_array = np.zeros(num_locations)
lamb_endo_mg1_array = np.zeros(num_locations)
wait_endo_mg1_array = np.zeros(num_locations)


rho_pooling_array = np.zeros(num_locations)
radius_pooling_array = np.zeros(num_locations)
t_pooling_array = np.zeros(num_locations)
obj_pooling_array = np.zeros(num_locations)
lamb_endo_pooling_array = np.zeros(num_locations)
wait_endo_pooling_array = np.zeros(num_locations)

rho_opt_array = np.zeros(num_locations)
obj_opt_array = np.zeros(num_locations)
mode_opt_array = -77*np.ones(num_locations)




for i in range(num_locations):
    
    print('**************************************** Iter',i,'**************************************')

    f_x_now = (f_x_values_peak[i], f_x_values_peak[i])

#======= Within Radius =============    
    Solution, obj_array = solve_mg1_radius(f_x_now, s_c_array[i])
    
    rho_mg1_array[i] = Solution['rho_opt']
    radius_mg1_array[i] = Solution['radius_opt']
    obj_mg1_array[i] = Solution['obj_opt']
    lamb_endo_mg1_array[i] = Solution['lamb_endo']
    wait_endo_mg1_array[i] = Solution['wait_endo']

#======= Pooling ===============
    Solution, obj_array = solve_pooling(f_x_now, s_c_array[i])
    
    rho_pooling_array[i] = Solution['rho_opt']
    radius_pooling_array[i] = Solution['radius_opt']
    t_pooling_array[i] = Solution['t0_opt']    
    obj_pooling_array[i] = Solution['obj_opt']
    lamb_endo_pooling_array[i] = Solution['lamb_endo']
    wait_endo_pooling_array[i] = Solution['wait_endo']

    
    who_is_best = np.argmax([obj_mg1_array[i], obj_pooling_array[i]])
    
    if who_is_best == 0:
        rho_opt_array[i] = rho_mg1_array[i]
        obj_opt_array[i] = obj_mg1_array[i]
        mode_opt_array[i] = 0
    elif who_is_best == 1:
        rho_opt_array[i] = rho_pooling_array[i]
        obj_opt_array[i] = obj_pooling_array[i]
        mode_opt_array[i] = 1


 

In [ ]:
solution_toronto = zipcode_pop_toronto.with_columns('f_x_peak',f_x_values_peak,
                                                    'lambda0', f_x_values_peak/half_day,
                                'rho_mg1_array',rho_mg1_array,
                                'radius_mg1_array',radius_mg1_array,
                                'obj_mg1_array', obj_mg1_array,
                                'lamb_endo_mg1_array', lamb_endo_mg1_array,
                                'wait_endo_mg1_array', wait_endo_mg1_array,
                                'rho_pooling_array', rho_pooling_array,
                                'radius_pooling_array', radius_pooling_array,
                                't_pooling_array', t_pooling_array,
                                'obj_pooling_array', obj_pooling_array,
                                'lamb_endo_pooling_array', lamb_endo_pooling_array,
                                'wait_endo_pooling_array', wait_endo_pooling_array,
                                'mode_opt_array', mode_opt_array,
                                'maximum profit', np.maximum(obj_mg1_array, obj_pooling_array),
                                'choice prob', np.maximum(lamb_endo_pooling_array,lamb_endo_mg1_array)/(f_x_values_peak/half_day))
solution_toronto.to_csv('solution_toronto_slower_shopping.csv')

In [ ]:
solution_toronto = Table.read_table('solution_toronto.csv')
#solution_toronto = Table.read_table('solution_toronto_less_willingness_to_wait.csv')
#solution_toronto = Table.read_table('solution_toronto_less_willingness_to_walk.csv')
#solution_toronto = Table.read_table('solution_toronto_slower_shopping.csv')

solution_toronto

In [ ]:
df_pop = solution_toronto.select('zipcode','maximum profit','mode_opt_array'
                       ).relabeled('zipcode','zip_code'
                                  ).relabeled('maximum profit','val').relabeled('mode_opt_array', 'mode').to_df()

In [ ]:
import folium

map_toronto_dt = folium.Map(location=[43.6548, -79.3883], zoom_start=11)

# marker = folium.Marker([min_lat, min_lon])
# map_toronto_dt.add_child(marker)
# marker = folium.Marker([min_lat, max_lon])
# map_toronto_dt.add_child(marker)
# marker = folium.Marker([max_lat, min_lon])
# map_toronto_dt.add_child(marker)
# marker = folium.Marker([max_lat, max_lon])
# map_toronto_dt.add_child(marker)

tor_dt_geo = "./toronto_dt.geojson"
choropleth_dt = folium.Choropleth(geo_data=tor_dt_geo,
    data = df_pop,
    #columns=['zip_code','val'],
    columns=['zip_code','mode'],   # uncomment this for identifying the modes
    key_on='feature.properties.CFSAUID',
    #threshold_scale = np.arange(0,18000,2000),
    fill_color='YlGn',
    fill_opacity=0.5, 
    line_opacity=0.3,
    legend_name='Optimal profit density').add_to(map_toronto_dt)

# add labels indicating the name of the community
style_function = "font-size: 15px; font-weight: bold"
choropleth_dt.geojson.add_child(
    folium.features.GeoJsonTooltip(['CFSAUID'], style=style_function, labels=False))
    
map_toronto_dt

In [ ]:
import os
import time
from selenium import webdriver

delay=5
fn='tor_scenario2_binary_YlGn_2022.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
map_toronto_dt.save(fn)

from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(tmpurl)

In [ ]:
#solution_toronto = Table.read_table('solution_toronto.csv')
#solution_toronto = Table.read_table('solution_toronto_less_willingness_to_wait.csv')
#solution_toronto = Table.read_table('solution_toronto_less_willingness_to_walk.csv')
solution_toronto = Table.read_table('solution_toronto_slower_shopping.csv')
solution_toronto

In [ ]:
wait_opt = np.zeros(36)
r_opt = np.zeros(36)

for i in range(36):
    if solution_toronto.column('mode_opt_array')[i] == 1:
        wait_opt[i] = solution_toronto.column('wait_endo_pooling_array')[i]
        r_opt[i] = solution_toronto.column('radius_pooling_array')[i]

    else:
        wait_opt[i] = solution_toronto.column('wait_endo_mg1_array')[i]
        r_opt[i] = solution_toronto.column('radius_mg1_array')[i]


        

In [ ]:
print(np.mean(wait_opt), np.mean(r_opt), np.mean(solution_toronto.column('maximum profit')), np.sum(solution_toronto.column('mode_opt_array')))  # baseline

In [ ]:
print(np.mean(wait_opt), np.mean(r_opt), np.mean(solution_toronto.column('maximum profit')), np.sum(solution_toronto.column('mode_opt_array')))

In [ ]:
# less willing to walk
print(np.mean(wait_opt), np.mean(r_opt), np.mean(solution_toronto.column('maximum profit')), np.sum(solution_toronto.column('mode_opt_array')))

In [ ]:
#longer shopping
print(np.mean(wait_opt), np.mean(r_opt), np.mean(solution_toronto.column('maximum profit')), np.sum(solution_toronto.column('mode_opt_array')))